In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.optimize as opt
%matplotlib inline

In [2]:
# base_dir = 'D:\\Src\\ensemble-forecasting'  # work
base_dir = '/media/iterater/DATA/_SRC_/ensemble-forecasting'  # home Ubuntu
h = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_hiromb_GI_60x434.txt')) - 37.356
s = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_swan_GI_48x434.txt'))
n = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_noswan_GI_48x434.txt'))
m = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_measurements_GI_2623.txt'))

# forecast dimensions
N = 430
T = 48

# preparing measurements forecast
m_fc = np.zeros((N, T+1))
for i in range(N):
    for j in range(T+1):
        m_fc[i, j] = m[i*6+j]

# data cut and shifting
shift_const = 0
h = h[0:N, 1:T+1] + shift_const
s = s[0:N, 1:T+1] + shift_const
n = n[0:N, 1:T+1] + shift_const
m_fc = m_fc[0:N, 1:T+1] + shift_const
h.shape, s.shape, n.shape, m_fc.shape

((430, 48), (430, 48), (430, 48), (430, 48))

In [28]:
ensembles_mask = np.array([[i & 1, (i >> 1) & 1, (i >> 2) & 1] for i in range(1, 8)])
def lin_comb(src, coeff):
    res = np.full(src[0].shape, coeff[-1])
    for i in range(len(src)):
        res += src[i] * coeff[i]
    return res
def get_coeff(mask, idcs):
    srcs = [pair[0][idcs[0]:idcs[1]] for pair in zip([h, s, n], mask) if pair[1]]
    res_coeff = np.zeros(4)
    res_coeff_pack = opt.minimize(lambda k: np.mean(np.abs(lin_comb(srcs, k) - m_fc[idcs[0]:idcs[1]])),
                                  np.full(sum(mask) + 1, 1.0, dtype=float)).x
    pos = np.where(mask)
    for i in range(len(pos)):
        res_coeff[pos[i]] = res_coeff_pack[i]
    res_coeff[-1] = res_coeff_pack[-1]
    return res_coeff
[get_coeff(msk, (0,N)) for msk in ensembles_mask]

[array([ 1.04341988,  0.        ,  0.        ,  4.2707991 ]),
 array([ 0.        ,  0.97928234,  0.        ,  2.27498968]),
 array([ 0.63596186,  0.63596186,  0.        ,  2.96451602]),
 array([ 0.        ,  0.        ,  1.05019829,  0.1644646 ]),
 array([ 0.63634796,  0.        ,  0.63634796,  1.74742268]),
 array([ 0.        ,  0.56571055,  0.56571055,  1.10209056]),
 array([ 0.61148834,  0.61148834,  0.61148834,  1.81959672])]

(array([0, 1]),)